In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt
import time

# 데이터 불러오기
(train_images, _), (_, _) = tf.keras.datasets.mnist.load_data()
train_images = train_images.reshape(train_images.shape[0], 28, 28, 1).astype('float32')
train_images = (train_images - 127.5) / 127.5  # [-1, 1]

BUFFER_SIZE = 60000
BATCH_SIZE = 256

train_dataset = tf.data.Dataset.from_tensor_slices(train_images).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

# 생성자 모델
def make_generator_model():
    model = tf.keras.Sequential([
        layers.Dense(7*7*256, use_bias=False, input_shape=(128,)),
        layers.BatchNormalization(),
        layers.LeakyReLU(),
        layers.Reshape((7, 7, 256)),
        layers.Conv2DTranspose(512, (5, 5), strides=(1, 1), padding='same', use_bias=False),
        layers.BatchNormalization(),
        layers.LeakyReLU(),
        layers.Conv2DTranspose(256, (5, 5), strides=(2, 2), padding='same', use_bias=False),
        layers.BatchNormalization(),
        layers.LeakyReLU(),
        layers.Conv2DTranspose(128, (5, 5), strides=(1, 1), padding='same', use_bias=False),
        layers.BatchNormalization(),
        layers.LeakyReLU(),
        layers.Conv2DTranspose(64, (5, 5), strides=(1, 1), padding='same', use_bias=False),
        layers.BatchNormalization(),
        layers.LeakyReLU(),
        layers.Conv2DTranspose(1, (5, 5), strides=(2, 2), padding='same', use_bias=False, activation='tanh')
    ])
    return model

# 판별자 모델
def make_discriminator_model():
    model = tf.keras.Sequential([
        layers.Conv2D(64, (5, 5), strides=(2, 2), padding='same', input_shape=[28, 28, 1]),
        layers.LeakyReLU(),
        layers.Dropout(0.3),
        layers.Conv2D(128, (5, 5), strides=(2, 2), padding='same'),
        layers.LeakyReLU(),
        layers.Dropout(0.3),
        layers.Flatten(),
        layers.Dense(1)
    ])
    return model

generator = make_generator_model()
discriminator = make_discriminator_model()

# 손실 함수 & 옵티마이저
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)
generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)

# 손실 함수 정의
def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    return real_loss + fake_loss

# 학습
EPOCHS = 100
noise_dim = 128
num_examples_to_generate = 16
seed = tf.random.normal([num_examples_to_generate, noise_dim])

@tf.function
def train_step(images):
    noise = tf.random.normal([BATCH_SIZE, noise_dim])

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_images = generator(noise, training=True)
        real_output = discriminator(images, training=True)
        fake_output = discriminator(generated_images, training=True)
        gen_loss = generator_loss(fake_output)
        disc_loss = discriminator_loss(real_output, fake_output)

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

def generate_and_save_images(model, epoch, test_input):
    predictions = model(test_input, training=False)
    predictions = (predictions + 1.0) / 2.0  # [-1,1] → [0,1]
    fig = plt.figure(figsize=(4, 4))
    for i in range(predictions.shape[0]):
        plt.subplot(4, 4, i + 1)
        plt.imshow(predictions[i, :, :, 0], cmap='gray')
        plt.axis('off')
    plt.suptitle(f'Epoch {epoch}')
    plt.tight_layout()
    plt.show()

def train(dataset, epochs):
    for epoch in range(1, epochs + 1):
        start = time.time()
        for image_batch in dataset:
            train_step(image_batch)

        if epoch % 10 == 0:
            generate_and_save_images(generator, epoch, seed)
        print(f'Epoch {epoch} done in {time.time()-start:.2f}s')

train(train_dataset, EPOCHS)


Epoch 1 done in 54.47s
Epoch 2 done in 47.81s


KeyboardInterrupt: 

In [ ]:
# Discriminator 평가용으로 컴파일
discriminator.compile(optimizer='adam',
                      loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                      metrics=['accuracy'])

# Real 이미지 (랜덤 샘플링)
real_images = train_images[np.random.randint(0, train_images.shape[0], BATCH_SIZE)]
real_labels = np.ones((BATCH_SIZE, 1))  # Real = 1

# Fake 이미지
noise = tf.random.normal([BATCH_SIZE, noise_dim])
generated_images = generator(noise, training=False)
fake_labels = np.zeros((BATCH_SIZE, 1))  # Fake = 0

# Discriminator 평가
d_loss_real = discriminator.evaluate(real_images, real_labels, verbose=0)
d_loss_fake = discriminator.evaluate(generated_images, fake_labels, verbose=0)

print(f"Discriminator Accuracy on Real Images: {d_loss_real[1] * 100:.2f}%")
print(f"Discriminator Accuracy on Fake Images: {d_loss_fake[1] * 100:.2f}%")


Discriminator Accuracy on Real Images: 27.34%
Discriminator Accuracy on Fake Images: 98.05%


"""Explanation"""


*   Discriminator Accuracy on Real Images: 27.34%
-> 진짜 이미지에 대한 정답은 "1"이나, 판별자가 그중 약 27% 만 진짜다 라고 맞춘 상태로
즉, real image 를 real 이라고 맞추지 못하고 있음

*   Discriminator Accuracy on Fake Images: 98.05%
-> 가짜 이미지에 대한 정답은 '0'으로, 판별자가 그중 98% 를 잘 맞추고 있다는 뜻으로 가짜를 잘 구분해 내고 있다는 것


즉, 판별기가 가짜는 잘 구별하지만, 진짜를 추론하는 과정에서는 약한 모습을 가지고 있는 모델임.
즉, 판별기의 성능이 더 좋아서 가짜 이미지를 잘 걸러내는 반면, 생성기의  성능은 좋지 않아서 진짜 같은 이미지를 만들어내고 있지 못하고 있는 상태

<해결방법 TRY- Genarator>

1. generator 레이어 조정

Conv2DTranspose 채널 수 조정


현재구조 [256->128->64->1]

수정구조 [512->256->128->64->1]

2. Noise 값 증가


초기값 128 에서 256으로 증가시킴


